In [1]:
NETWORK_MAGIC = b'\xf9\xbe\xb4\xd9'
TESTNET_NETWORK_MAGIC = b'\x0b\x11\x09\x07'


class NetworkEnvelope:

    def __init__(self, command, payload, testnet=False):
        self.command = command
        self.payload = payload
        if testnet:
            self.magic = TESTNET_NETWORK_MAGIC
        else:
            self.magic = NETWORK_MAGIC

    def __repr__(self):
        return '{}: {}'.format(
            self.command.decode('ascii'),
            self.payload.hex(),
        )

# 연습문제 10.1 위 클래스의 parse 클래스 메서드를 작성하시오

In [2]:
@classmethod
def parse(cls,s,testnet=False):
    magic = s.read(4)
    if magic==b'':
        raise IOError('Connection reset!')
    if testnet:
        expected_magic = TESTNET_NETWORK_MAGIC
    else:
        expected_magic=NETWORK_MAGIC
    if magic !=expected_magic:
        raise SyntaxError('magic is not right {} vs {}'.format(magic.hex(),expected_magic.hex()))
    command = s.read(12)
    command = command.strip(b'\x00')
    payload_length = little_endian_to_int(s.read(4))
    checksum=s.read(4)
    payload = s.read(payload_length)
    calculated_checksum = hash256(payload)[:4]
    if calculated_checksum !=checksum:
        raise IOError('checksum does not match')
    return cls(command,payload, testnet=testnet)

# 연습문제 10.2 아래는 어떤 네트워크 메시지인가? 
- f9beb4d976657261636b000000000000000000005df6e0e2

In [3]:
class NetworkEnvelope:
    from network import NetworkEnvelope
    from io import BytesIO
    message_hex = 'f9beb4d976657261636b000000000000000000005df6e0e2'
    stream = BytesIO(bytes.fromhex(message_hex))
    envelope = NetworkEnvelope.parse(stream)
    print(envelope.command)

b'verack'


# 연습문제 10.3 networkenvelope 클래스의 serialize 메서드를 작성하시오.

In [4]:
class NetworkEnvelope:
    def serialize(self):
        result = self.magic
        result +=self.command + b'\x00'*(12-len(self.command))
        result +=int_to_little_endian(len(self.payload),4)
        result += hash256(self.payload)[:4]
        result += self.payload
        return result

In [5]:
class VersionMessage:
    command = b'version'

def __init__(self, version =70015, services=0, timestamp=None,
             receiver_services=0,
             receiver_ip=b'\x00\x00\x00\x00', receiver_port=8333,
             sender_services=0,
             sender_ip=b'\x00\x00\x00\x00',sender_port=8333,
             nonce=None,user_agent=b'/programmingbitcoin:0.1/',
             latest_block=0,relay=False):
    self.version = version
    self.services = services
    if timestamp is None:
        self.timestamp = int(time.time())
    else:
        self.timestamp = timestamp
    self.receiver_services = receiver_services
    self.receiver_ip = receiver_ip
    self.receiver_port = receiver_port
    self.sender_services = sender_services
    self.sender_ip=sender_ip
    self.sender_port=sender_port
    if nonce is None:
        self.nonce = int_to_little_endian(randint(0,2**64),8)
    else:
        self.nonce=nonce
    self.user_agent=user_agent
    self.latest_block=latest_block
    self.relay=relay

# 연습문제 10.4 versionmessage 클래스의 serilize 메서드를 작성하시오

In [6]:
class VersionMessage:
    def __init__(self, version =70015, services=0, timestamp=None,
             receiver_services=0,
             receiver_ip=b'\x00\x00\x00\x00', receiver_port=8333,
             sender_services=0,
             sender_ip=b'\x00\x00\x00\x00',sender_port=8333,
             nonce=None,user_agent=b'/programmingbitcoin:0.1/',
             latest_block=0,relay=False):
        self.version = version
        self.services = services
        if timestamp is None:
            self.timestamp = int(time.time())
        else:
            self.timestamp = timestamp
        self.receiver_services = receiver_services
        self.receiver_ip = receiver_ip
        self.receiver_port = receiver_port
        self.sender_services = sender_services
        self.sender_ip=sender_ip
        self.sender_port=sender_port
        if nonce is None:
            self.nonce = int_to_little_endian(randint(0,2**64),8)
        else:
            self.nonce=nonce
        self.user_agent=user_agent
        self.latest_block=latest_block
        self.relay=relay
    def serialize(self):
        result = int_to_little_endian(self.version, 4)
        result += int_to_little_endian(self.services, 8)
        result += int_to_little_endian(self.timestamp, 8)
        result += int_to_little_endian(self.receiver_services, 8)
        result += b'\x00' * 10 + b'\xff\xff' + self.receiver_ip
        result += self.receiver_port.to_bytes(2, 'big')
        result += int_to_little_endian(self.sender_services, 8)
        result += b'\x00' * 10 + b'\xff\xff' + self.sender_ip
        result += self.sender_port.to_bytes(2, 'big')
        result += self.nonce
        result += encode_varint(len(self.user_agent))
        result += self.user_agent
        result += int_to_little_endian(self.latest_block, 4)
        if self.relay:
            result += b'\x01'
        else:
            result += b'\x00'
        return result
    


In [7]:
import socket
from network import NetworkEnvelope, VersionMessage
host = 'testnet.programmingbitcoin.com'
port = 18333
socket = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
socket.connect((host,port))
stream = socket.makefile('rb',None)
version = VersionMessage()
envelope = NetworkEnvelope(version.command, version.serialize())
socket.sendall(envelope.serialize())
while True:
    new_message = NetworkEnvelope.parse(stream)
    print(new_message)

TimeoutError: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다

In [8]:
class VerAckMessage:
    command = b'verack'

    def __init__(self):
        pass

    @classmethod
    def parse(cls, s):
        return cls()
    def serialize(self):
        return b''

In [9]:
class SimpleNode:

    def __init__(self, host, port=None, testnet=False, logging=False):
        if port is None:
            if testnet:
                port = 18333
            else:
                port = 8333
        self.testnet = testnet
        self.logging = logging
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.socket.connect((host, port))
        self.stream = self.socket.makefile('rb', None)
    

    def handshake(self):
        '''Do a handshake with the other node.
        Handshake is sending a version message and getting a verack back.'''
        # create a version message
        # send the command
        # wait for a verack message
        raise NotImplementedError
   

    def send(self, message):  # <1>
        '''Send a message to the connected node'''
        envelope = NetworkEnvelope(
            message.command, message.serialize(), testnet=self.testnet)
        if self.logging:
            print('sending: {}'.format(envelope))
        self.socket.sendall(envelope.serialize())

    def read(self):  # <2>
        '''Read a message from the socket'''
        envelope = NetworkEnvelope.parse(self.stream, testnet=self.testnet)
        if self.logging:
            print('receiving: {}'.format(envelope))
        return envelope

    def wait_for(self, *message_classes):  # <3>
        '''Wait for one of the messages in the list'''
        command = None
        command_to_class = {m.command: m for m in message_classes}
        while command not in command_to_class.keys():
            envelope = self.read()
            command = envelope.command
            if command == VersionMessage.command:
                self.send(VerAckMessage())
            elif command == PingMessage.command:
                self.send(PongMessage(envelope.payload))
        return command_to_class[command].parse(envelope.stream())

In [10]:
from network import SimpleNode, VersionMessage
node = SimpleNode('testnet.programmingbitcoin.com',testnet=True)
version = VersionMessage()
node.send(version)
verack_received=False
version_received=False
while not verack_received or not version_received:
    message = node.wait_for(VersionMessage,VerAckMessage)
    if message.command == VerAckMessage.command:
        verack_received = True
    else:
        version_received=True

KeyboardInterrupt: 

# 연습문제 10.5 simplenode 클래스의 handshake 메서드를 작성하시오.

In [11]:
class SimpleNode:

    def handshake(self):
        version = VersionMessage()
        self.send(version)
        verack_received = True
        version_received = True
        while not (verack_received and version_received):
            message = self.wait_for(VerAckMessage, VersionMessage)
            verack_received = message.command == VerAckMessage.command
            version_received = message.command == VersionMessage.command

In [12]:
class GetHeadersMessage:
    command = b'getheaders'

    def __init__(self, version =70015, num_hashes=1,
                 start_block=None,end_block=None):
        self.version = version
        self.num_hashes = num_hashes
        if start_block is None:
            raise RuntimeError('a start block is required')
        self.start_block = start_block
        if end_block is None:
            self.end_block = b'\x00'*32
        else:
            self.end_block = end_block

# 연습문제 10.6 GetHeadersMessage 클래스의 serialize 메서드를 작성하시오.

In [13]:
class GetHeadersMessage:
    def serialize(self):
        result = int_to_little_endian(self.version,4)
        result += encode_varint(self.num_hashes)
        result += self.start_block[::-1]
        result += self.end_block[::-1]
        return result

In [14]:
from io import BytesIO
from block import Block, GENESIS_BLOCK
from network import SimpleNode, GetHeadersMessage
node = SimpleNode('mainnet.programmingbitcoin.com', testnet=False)
node.handshake()
genesis = Block.parse(BytesIO(GENESIS_BLOCK))
getheaders=GetHeadersMessage(start_block=genesis.hash())
node.send(getheaders)

KeyboardInterrupt: 

In [15]:
class HeadersMessage:
    command = b'headers'

    def __init__(self, blocks):
        self.blocks == blocks

    @classmethod
    def parse(cls, stream):
        num_headers = read_varint(stream)
        blocks =[]
        for _ in range(num_headers):
            blcoks.append(Block.parse(stream))
            num_txs = read_varint(stream)
            if num_txs !=0:
                raise RuntimeError('number of txs not 0')
        return cls(blocks)

In [16]:
>>> from io import BytesIO
>>> from network import SimpleNode, GetHeadersMessage, HeadersMessage
>>> from block import Block, GENESIS_BLOCK, LOWEST_BITS
>>> from helper import calculate_new_bits
>>> previous = Block.parse(BytesIO(GENESIS_BLOCK))
>>> first_epoch_timestamp = previous.timestamp
>>> expected_bits = LOWEST_BITS
>>> count = 1
>>> node = SimpleNode('mainnet.programmingbitcoin.com', testnet=False)
>>> node.handshake()
>>> for _ in range(19):
...     getheaders = GetHeadersMessage(start_block=previous.hash())
...     node.send(getheaders)
...     headers = node.wait_for(HeadersMessage)
...     for header in headers.blocks: # 작업 증명이 올바른지 확인한다.
...         if not header.check_pow():  # <1>
...             raise RuntimeError('bad PoW at block {}'.format(count))
...         if header.prev_block != previous.hash():  # <2> 현재 블록이 직전블록의 다음 블록인지 확인한다.
...             raise RuntimeError('discontinuous block at {}'.format(count))
...         if count % 2016 == 0:
...             time_diff = previous.timestamp - first_epoch_timestamp
...             expected_bits = calculate_new_bits(previous.bits, time_diff)  # <4> 매 2016블록마다 새로운 비트값을 계산한다.
...             print(expected_bits.hex())
...             first_epoch_timestamp = header.timestamp  # <5> 실제 걸린 시간을 계산하기 위해 필요한 현 블록헤더의 타임스탬프를 저장한다.
...         if header.bits != expected_bits:  # <3> 공식에 따라 계산한 비트값이 실제 적혀 있는 값과 동일한지 확인한다.
...             raise RuntimeError('bad bits at block {}'.format(count))
...         previous = header
...         count += 1

TimeoutError: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다

In [17]:
from helper import hash256
hash0 =bytes.fromhex('c117ea8ec828342f4dfb0ad6bd140e03a50720ece40169ee38b\
dc15d9eb64cf5')
hash1=bytes.fromhex('c131474164b412e3406696da1ee20ab0fc9bf41c8f05fa8ceea\
7a08d672d7cc5')

In [18]:
hash0

b'\xc1\x17\xea\x8e\xc8(4/M\xfb\n\xd6\xbd\x14\x0e\x03\xa5\x07 \xec\xe4\x01i\xee8\xbd\xc1]\x9e\xb6L\xf5'

In [19]:
parent= hash256(hash0+hash1)
print(parent.hex())

8b30c5ba100f6f2e5ad1e2a742e5020491240f8eb514fe97c713c31718ad7ecd


# 11.4 머클부모 레벨

In [22]:
from helper import merkle_parent
hex_hashes=[
    'c117ea8ec828342f4dfb0ad6bd140e03a50720ece40169ee38bdc15d9eb64cf5',
    'c131474164b412e3406696da1ee20ab0fc9bf41c8f05fa8ceea7a08d672d7cc5',
    'f391da6ecfeed1814efae39e7fcb3939ae0b02c02ae7d0a5848a66947c0727b0',
    '3d238a92a94532b946c90e19c49351c763696cff3db400485b813aecb8a13181',
    '10092f2633be5f3ce349bf9ddbde36caa3dd10dfa0ec8106bce23acbff637dae',
    
]


In [23]:
hashes =[bytes.fromhex(x) for x in hex_hashes]
if len(hashes) % 2==1:
    hashes.append(hashes[-1])
parent_level=[]
for i in range(0,len(hashes),2):
    parent = merkle_parent(hashes[i],hashes[i+1])
    parent_level.append(parent)
for item in parent_level:
    print(item.hex())

8b30c5ba100f6f2e5ad1e2a742e5020491240f8eb514fe97c713c31718ad7ecd
4ffdfadd5cabd22d6a5c81aeb8d8253db01bc990a8898b62b6d6e0414ade260c
3ecf6115380c77e8aae56660f5634982ee897351ba906a6837d15ebc3a225df0


# 11.5. 머클루트

In [30]:
def merkle_parent_level(hashes):
    if len(hashes)==1:
        raise RuntimeError('Cannot take parent level')
    if len(hashes)%2==1:
        hashes.append(hashes[-1])
    parent_level=[]
    for i in range(0,len(hashes),2):
        parent = merkle_parent(hashes[i],hashes[i+1])
        parent_level.append(parent)
    return parent_level

In [27]:
# from helper import merkle_parent_level
hex_hashes=[
    'c117ea8ec828342f4dfb0ad6bd140e03a50720ece40169ee38bdc15d9eb64cf5',
    'c131474164b412e3406696da1ee20ab0fc9bf41c8f05fa8ceea7a08d672d7cc5',
    'f391da6ecfeed1814efae39e7fcb3939ae0b02c02ae7d0a5848a66947c0727b0',
    '3d238a92a94532b946c90e19c49351c763696cff3db400485b813aecb8a13181',
    '10092f2633be5f3ce349bf9ddbde36caa3dd10dfa0ec8106bce23acbff637dae',
    '7d37b3d54fa6a64869084bfd2e831309118b9e833610e6228adacdbd1b4ba161',
    '8118a77e542892fe15ae3fc771a4abfd2f5d5d5997544c3487ac36b5c85170fc',
    'dff6879848c2c9b62fe652720b8df5272093acfaa45a43cdb3696fe2466a3877',
    'b825c0745f46ac58f7d3759e6dc535a1fec7820377f24d4c2c6ad2cc55c0cb59',
    '95513952a04bd8992721e9b7e2937f1c04ba31e0469fbe615a78197f68f52b7c',
    '2e6d722e5e4dbdf2447ddecc9f7dabb8e299bae921c99ad5b0184cd9eb8e5908',
    'b13a750047bc0bdceb2473e5fe488c2596d7a7124b4e716fdd29b046ef99bbf0',

]

In [31]:
hashes =[bytes.fromhex(x) for x in hex_hashes]
current_hashes = hashes
while len(current_hashes)>1:
    current_hashes = merkle_parent_level(current_hashes)
print(current_hashes[0].hex())

020987f024f020be3858f4ddc17b52f81010c9941a4f67799235e8e55e68ba66


In [32]:
def merkle_root(hashes):
    current_level=hashes
    while len(current_level)>1:
        current_level = merkle_parent_level(current_level)
    return current_level[0]

In [34]:

tx_hex_hashes = [
    'c117ea8ec828342f4dfb0ad6bd140e03a50720ece40169ee38bdc15d9eb64cf5',
    'c131474164b412e3406696da1ee20ab0fc9bf41c8f05fa8ceea7a08d672d7cc5',
    'f391da6ecfeed1814efae39e7fcb3939ae0b02c02ae7d0a5848a66947c0727b0',
    '3d238a92a94532b946c90e19c49351c763696cff3db400485b813aecb8a13181',
    '10092f2633be5f3ce349bf9ddbde36caa3dd10dfa0ec8106bce23acbff637dae',
    '7d37b3d54fa6a64869084bfd2e831309118b9e833610e6228adacdbd1b4ba161',
    '8118a77e542892fe15ae3fc771a4abfd2f5d5d5997544c3487ac36b5c85170fc',
]
tx_hashes = [bytes.fromhex(x) for x in tx_hex_hashes]
hashes = [h[::-1] for h in tx_hashes]
print(merkle_root(hashes)[::-1].hex())

cef9d1f139ff094cfeb60ac4df835e70a2eceb17fc7efc7caf58cd9f9033dc15


In [35]:
class Block:
    def __init__(self, version,prev_block, merkel_root,
                 timestamp,bits,nonce, tx_hashes=None):
        self.version = version
        self.prev_block = prev_block
        self.merkle_root = merkle_root
        self.timestamp = timestamp
        self.bits=bits
        self.nonce=nonce
        self.tx_hashes=tx_hashes

# 연습문제 11.4 Block 클래스에서 validate_merkle_root 메서드를 작성하시오.

In [36]:
class Block:
    def validate_merkle_root(self):
        hashes = [h[::-1] for h in self.tx_hashes]
        root = merkle_root(hashes)
        return root[::-1]==self.merkle_root

# 연습문제 11.5. 27개의 단말 노드가 달린 머클트리를 구성하고 모든 노드를 None 값으로 초기화하시오. 그리고 머클트리의 각 레벨을 출력하시오.

In [37]:
import math
total = 27
max_depth = math.ceil(math.log(total,2))
merkle_tree=[]
for depth in range(max_depth+1):
    num_items = math.ceil(total / 2**(max_depth-depth))
    level_hashes = [None]*num_items
    merkle_tree.append(level_hashes)
for level in merkle_tree:
    print(level)

[None]
[None, None]
[None, None, None, None]
[None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


# 연습문제 11.6. MerkleBlcok 클래스의 parse 메서드를 작성하시오

In [1]:
class MerkleBlock:
    @classmethod
    def parse(cls,s):
        version = little_endian_to_int(s.read(4))
        prev_block=s.read(32)[::-1]
        merkle_root=s.read(32)[::-1]
        timestamp = little_endian_to_int(s.read(4))
        bits = s.read(4)
        nonce=s.read(4)
        total = little_endian_to_int(s.read(4))
        num_hashes = read_varint(s)
        hashes = []
        for _ in range(num_hashes):
            hashes.append(s.read(32)[::-1])
        flags_length = read_varint(s)
        flags = s.read(flags_length)
        return cls(version, prev_block, merkle_root, timestamp, bits,
        nonce,total,hashes,flags)
        

# 연습문제 11.7. MerkleBlcok 클래스의 is_valid 메서드를 작성하시오

In [2]:
class MerkleBlock:
    def is_valid(self):
        flag_bits = bytes_to_bit_field(self.flags)
        hashes=[h[::-1]for h in self.hashes]
        merkle_tree = MerkleTree(self.total)
        merkle_tree.populate_tree(flag_bits,hashes)
        return merkle_tree.root()[::-1]==self.merkle_root